# Load libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.base import clone
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFE, RFECV, SelectFromModel, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures

# Load data

In [3]:
X, y, feature_names, _, _ = load_boston().values()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X, columns = feature_names), pd.Series(y, name='label'), test_size = .2, random_state=42)

In [5]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train: Linear Regression

In [6]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

pred_lr = model_lr.predict(X_test)

print(model_lr.score(X_test, y_test))
print(mean_absolute_error(y_test, pred_lr))
print(mean_absolute_percentage_error(y_test, pred_lr))

0.668759493535632
3.1890919658878483
0.16866394539378712


In [24]:
pf = PolynomialFeatures(interaction_only=True, include_bias=False, degree=2)

X_train = pf.fit_transform(X_train)
X_test = pf.transform(X_test)

In [99]:
rfe = RFECV(estimator=model_lr, cv=5, scoring='r2')

X_train = rfe.fit_transform(X_train, y_train)
X_test = rfe.transform(X_test)

In [100]:
model_lr.fit(X_train, y_train)

pred_lr = model_lr.predict(X_test)

print(model_lr.score(X_test, y_test))
print(mean_absolute_error(y_test, pred_lr))
print(mean_absolute_percentage_error(y_test, pred_lr))

0.7635982049770087
3.7379107553033477
0.18737837457744916


# Train: Random Forest

In [7]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X, columns = feature_names), pd.Series(y, name='label'), test_size = .2, random_state=42)

In [8]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

model.score(X_test, y_test, )

0.8920995891343227

In [38]:
pf = PolynomialFeatures(interaction_only=True, include_bias=False, degree=2)

X_train = pf.fit_transform(X_train)
X_test = pf.transform(X_test)

In [39]:
rfe = RFECV(estimator=model, cv=5, scoring='r2')

X_train = rfe.fit_transform(X_train, y_train)
X_test = rfe.transform(X_test)

In [40]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

model.score(X_test, y_test, )

0.8556048645974019

## Recursive Feature Elimination

### by sklearn

In [145]:
rfe_ = RFE(estimator=model, n_features_to_select=5)

In [146]:
rfe_.fit(X_train, y_train)

rfe_.ranking_

array([1, 8, 6, 9, 3, 1, 4, 1, 7, 2, 1, 5, 1])

### implementation

In [140]:
class rfe:
    def __init__(self, estimator, n_select = None):
        self.estimator  = clone(estimator)
        self.n_select = n_select
        
    def fit(self, X, y):
        self.n_features_ori = X.shape[1]
        result = np.ones(self.n_features_ori, dtype=int)
        
        if self.n_select is None:
            self.n_features = self.n_features_ori//2
        else:
            self.n_features = self.n_select
        
        n = 0
        while n < self.n_features_ori - self.n_features:
            self.estimator.fit(X * np.array(result==1, dtype = int), y)
            try:
                rk = np.argsort(self.estimator.feature_importances_)
            except:
                rk = np.argsort(np.square(self.estimator.coef_.flatten()))
            
            min_idx = rk[:n+1]
            result[min_idx] += 1 
            n += 1
       
        self.ranking = result
        self.support = result == 1
        
        return None
    
    def transform(self, X):
        return X[:, self.support]
        
    def fit_transform(self, X, y):
        if y is None:
            return self.fit(X).transform(X)
        else:
            return self.fit(X, y).transform(X)

In [147]:
rfe0 = rfe(model, n_select=5)
rfe0.fit(X_train, y_train)

In [148]:
rfe0.ranking

array([1, 8, 6, 9, 3, 1, 4, 1, 7, 2, 1, 5, 1])

### comparison

In [149]:
np.alltrue(rfe0.ranking == rfe_.ranking_)

True

In [150]:
np.alltrue(rfe0.transform(X_train)==rfe_.transform(X_train))

True

## Sequencial Feature Selection: forward and backward

### by sklearn

In [15]:
sfs_ = SequentialFeatureSelector(estimator=model_lr, direction='backward', )

In [16]:
sfs_.fit(X_train, y_train)

sfs_.support_

array([False, False, False, False,  True,  True, False,  True, False,
       False,  True,  True,  True])

### implementation

In [9]:
class sfs:
    def __init__(self, estimator, n_select = None, stop_when_best = False,cv = 5, direction = 'forward'):
        self.estimator = clone(estimator)
        self.n_select = n_select
        self.cv = cv
        self.direction = direction
        self.stop_when_best = stop_when_best
    
    def fit(self, X, y):
        self.n_features_ori = X.shape[1]
        if self.n_select is None:
            self.n_features = self.n_features_ori//2
        else:
            self.n_features = self.n_select
        if self.direction == 'backward':
            self.n_features = self.n_features_ori - self.n_features
        
        self.support = np.zeros(self.n_features_ori, dtype = bool)
        
        self.scores = np.array([])
        
        for _ in range(self.n_features):
            non_selected = np.flatnonzero(~self.support)
            dict_score = dict()
            for f in non_selected:
                candidates = self.support.copy()
                candidates[f] = True
                if self.direction == 'backward':
                    candidates = ~candidates
                cvs = cross_val_score(estimator=self.estimator, X = X[:, candidates], y= y, cv=self.cv).mean()
                dict_score[f] = cvs
            selected = max(dict_score, key=lambda x: dict_score[x])
            
            self.scores = np.append(self.scores, dict_score[selected])
            
            if self.stop_when_best:
                if self.scores.size<2:
                    self.support[selected] = True
                else:
                    if self.scores[-1] > self.scores[-2]:
                        self.support[selected] = True
                    else:
                        break
            else:
                self.support[selected] = True
            
        self.best_n_select = np.argmax(self.scores) + 1
        if self.direction == 'backward':
            self.support = ~self.support
            
        return None
    
    def transform(self, X):
        return X[:, self.support]
    
    def fit_transform(self, X, y):
        if y is None:
            return self.fit(X).transform(X)
        else:
            return self.fit(X, y).transform(X)

In [12]:
sfs0 = sfs(estimator=model_lr, direction='backward', )
sfs0.fit(X_train, y_train)

sfs0.support

array([False, False, False, False,  True,  True, False,  True, False,
       False,  True,  True,  True])

### comparison: backward

In [17]:
np.alltrue(sfs_.support_ == sfs0.support)

True

In [18]:
np.alltrue(sfs_.transform(X_train)==sfs0.transform(X_train))

True

In [19]:
sfs0.scores

array([0.7256365 , 0.72601528, 0.72612216, 0.72353299, 0.71988332,
       0.71780413, 0.71291874])

In [37]:
sfs0.best_n_select

3

In [11]:
sfs1 = sfs(estimator=model_lr, direction='backward', stop_when_best= True)
sfs1.fit(X_train, y_train)

sfs1.support

array([ True, False, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True])

In [20]:
sfs1.scores

array([0.7256365 , 0.72601528, 0.72612216, 0.72353299])

### comparison: forward

In [16]:
sfs_ = SequentialFeatureSelector(estimator=model_lr, direction='forward', )
sfs_.fit(X_train, y_train)

sfs0 = sfs(estimator=model_lr, direction='forward', )
sfs0.fit(X_train, y_train)

np.alltrue(sfs_.support_ == sfs0.support)

True

## Select from model

### by sklearn

In [174]:
sfm = SelectFromModel(estimator=model, threshold='mean')

In [175]:
sfm.fit(X_train, y_train)

sfm.get_support()

array([False, False, False, False, False,  True, False, False, False,
       False, False, False,  True])

In [176]:
sfm.threshold_

0.07692307692307693

### implementation

In [183]:
class SFM:
    def __init__(self, estimator, strategy='mean'):
        self.estimator = clone(estimator)
        self.strategy = strategy
    
    def fit(self, X, y):
        self.estimator.fit(X, y)
        
        try:
            self.importance = self.estimator.feature_importances_
        except:
            self.importance = np.abs(self.estimator.coef_.flatten())
            
        if self.strategy == 'mean':
            self.threshold = self.importance.mean()
        elif self.strategy == 'median':
            self.threshold = np.median(self.importance)
        else:
            self.threshold = self.strategy.copy()
            
        self.support = self.importance >= self.threshold
        return None
    
    def transform(self, X):
        return X[:, self.support]
    
    def fit_transform(self, X, y):
        return self.fit(X, y).transform(X)

In [184]:
sfm0 = SFM(estimator=model, strategy='mean')

In [187]:
sfm0.fit(X_train, y_train)

sfm0.support

array([False, False, False, False, False,  True, False, False, False,
       False, False, False,  True])

### comparison

In [189]:
np.alltrue(sfm.get_support() == sfm0.support)

True

In [190]:
np.alltrue(sfm.transform(X_train) == sfm0.transform(X_train))

True